In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### The Main Idea behind this notebook is the specific feature engineering techniques used.
### This is an experimentation as to how much we can infer from the text without making custom embeddings.And Fitting an XGB Regressor with those extracted features.
### I hope you canfindsomething useful :). Happy Coding!

## Importing Libraries

In [ ]:
# ! pip install pyphen
# !pip download ../input/ pyphen 
!pip install /kaggle/input/textstat/Pyphen-0.10.0-py3-none-any.whl

In [ ]:
# ! pip install textstat
# !pip download textstat
!pip install /kaggle/input/textstat/textstat-0.7.0-py3-none-any.whl

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

import json
from collections import defaultdict
import pyphen
import textstat
import gc

gc.enable()
from IPython.display import clear_output
from tqdm import tqdm

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
from xgboost import XGBRegressor

## Loading Data

In [ ]:
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')

## Feature Engineering

In [ ]:
# Calculate numberof syllables per word
dic = pyphen.Pyphen(lang='en')
def getSylabblesperWord(word):
    sylables = dic.inserted(train.excerpt[0]).count("-") - word.count("-")
    return sylables / len(word.split(" "))

In [ ]:
train_pre = train.copy()
train_pre["Syllable_Ratio"] = train_pre["excerpt"].apply(getSylabblesperWord)

In [ ]:
train_pre

In [ ]:
def get_readability_scores(df):
    
    flesch_score = []
    smog_index = []
    flesch_kincaid = []
    coleman_liau = []
    automated_readability = []
    dale_chall_readability = []
    difficult_w_ratio = []
    linsear = []
    gunning_fog = []
    text_standard = []
    fernandez_huerta = []
    szigriszt_pazos = []
    gutierrez_polini = []
    crawford = []
       
    
    
    for sent in tqdm(df["excerpt"]):
        sent_len = len(sent.split(" "))
        
        
        flesch_score.append(textstat.flesch_reading_ease(sent))
        smog_index.append(textstat.smog_index(sent))
        flesch_kincaid.append(textstat.flesch_kincaid_grade(sent))
        coleman_liau.append(textstat.coleman_liau_index(sent))
        automated_readability.append(textstat.automated_readability_index(sent))
        dale_chall_readability.append(textstat.dale_chall_readability_score(sent))
        difficult_w_ratio.append(textstat.difficult_words(sent)/sent_len)
        linsear.append(textstat.linsear_write_formula(sent))
        gunning_fog.append(textstat.gunning_fog(sent))
        text_standard.append(textstat.text_standard(sent))
        fernandez_huerta.append(textstat.fernandez_huerta(sent))
        szigriszt_pazos.append(textstat.szigriszt_pazos(sent))
        gutierrez_polini.append(textstat.gutierrez_polini(sent))
        crawford.append(textstat.crawford(sent))
        
    
    flesch_score = np.array(flesch_score)
    smog_index = np.array(smog_index)
    flesch_kincaid = np.array(flesch_kincaid)
    coleman_liau = np.array(coleman_liau)
    automated_readability = np.array(automated_readability)
    dale_chall_readability = np.array(dale_chall_readability)
    difficult_w_ratio = np.array(difficult_w_ratio)
    linsear = np.array(linsear)
    gunning_fog = np.array(gunning_fog)
    text_standard = np.array(text_standard)
    fernandez_huerta = np.array(fernandez_huerta)
    szigriszt_pazos = np.array(szigriszt_pazos)
    gutierrez_polini = np.array(gutierrez_polini)
    crawford = np.array(crawford)
       
    
    return (flesch_score, smog_index, flesch_kincaid, coleman_liau, automated_readability, dale_chall_readability, 
            difficult_w_ratio, linsear, gunning_fog, text_standard, fernandez_huerta, szigriszt_pazos, gutierrez_polini,
            crawford)

In [ ]:
(flesch_score, smog_index, flesch_kincaid, coleman_liau, automated_readability, dale_chall_readability, 
            difficult_w_ratio, linsear, gunning_fog, text_standard, fernandez_huerta, szigriszt_pazos, gutierrez_polini,
            crawford) = get_readability_scores(train_pre)

In [ ]:
train_pre["flesch_score"] = flesch_score
train_pre["smog_index"] = smog_index
train_pre["flesch_kincaid"] = flesch_kincaid
train_pre["coleman_liau"] = coleman_liau
train_pre["automated_readability"] = automated_readability
train_pre["dale_chall_readability"] = dale_chall_readability
train_pre["difficult_w_ratio"] = difficult_w_ratio
train_pre["linsear"] = linsear
train_pre["gunning_fog"] = gunning_fog
train_pre["text_standard"] = text_standard
train_pre["fernandez_huerta"] = fernandez_huerta
train_pre["szigriszt_pazos"] = szigriszt_pazos
train_pre["gutierrez_polini"] = gutierrez_polini
train_pre["crawford"] = crawford

In [ ]:
train_pre["num_words"] = train_pre.excerpt.apply(lambda x : len(x.split( )))

In [ ]:
sns.heatmap(train_pre.corr());

In [ ]:
train_pre.columns

In [ ]:
def model_input_pipeline(df):
#     Feature Engineering
    df["Syllable_Ratio"] = df["excerpt"].apply(getSylabblesperWord)
    
    (flesch_score, smog_index, flesch_kincaid, coleman_liau, automated_readability, dale_chall_readability, 
            difficult_w_ratio, linsear, gunning_fog, text_standard, fernandez_huerta, szigriszt_pazos, gutierrez_polini,
            crawford) = get_readability_scores(df)
    
    df["flesch_score"] = flesch_score
    df["smog_index"] = smog_index
    df["flesch_kincaid"] = flesch_kincaid
    df["coleman_liau"] = coleman_liau
    df["automated_readability"] = automated_readability
    df["dale_chall_readability"] = dale_chall_readability
    df["difficult_w_ratio"] = difficult_w_ratio
    df["linsear"] = linsear
    df["gunning_fog"] = gunning_fog
    df["text_standard"] = text_standard
    df["fernandez_huerta"] = fernandez_huerta
    df["szigriszt_pazos"] = szigriszt_pazos
    df["gutierrez_polini"] = gutierrez_polini
    df["crawford"] = crawford
    
    df["num_words"] = df.excerpt.apply(lambda x : len(x.split( )))
    
    
    df = pd.get_dummies(df, columns=["text_standard"])
#     ["id", "target", "standard_error", "url_legal", "excerpt", "license"]
    if "standard_error" in df.columns:
        del df["standard_error"]
    if "licence" in df.columns:
        del df["license"]
        
    return df
    

In [ ]:
data = model_input_pipeline(train)

In [ ]:
X = data.drop(columns=["id", "target", "url_legal", "excerpt", "license"])
y = data.target

In [ ]:
train_cols = X.columns

In [ ]:
poly = PolynomialFeatures(2)
X = poly.fit_transform(X)

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.15, random_state = 123)

## XGB Regressor

In [ ]:
xgb_r = XGBRegressor( n_estimators = 15)
xgb_r.fit(train_X, train_y)

In [ ]:
pred = xgb_r.predict(test_X)

In [ ]:
rmse = np.sqrt(mean_squared_error(test_y, pred))
print("RMSE : % f" %(rmse))

In [ ]:
test_pre=test.copy()

In [ ]:
test_pre = model_input_pipeline(test_pre)

In [ ]:
test_X = test_pre.drop(columns=["id", "url_legal", "excerpt"])

In [ ]:
# To ensure all dummy columns are vailable
test_X = test_X.reindex(columns = train_cols)

In [ ]:
test_X = test_X.fillna(0)

In [ ]:
test_X = poly.fit_transform(test_X)

In [ ]:
pred = xgb_r.predict(test_X)

In [ ]:
submission_df = pd.DataFrame()

submission_df["id"] = test_pre["id"] 
submission_df["target"] = pred

In [ ]:
submission_df.to_csv('/kaggle/working/submission.csv', index=False)